In [3]:
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import date, timedelta
from pandas.tseries.offsets import *
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
xsl_path = "\\Users\\User\\OneDrive\\A6\\Excel\\"

year = 2025
quarter = 3

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2025, 8, 18), datetime.date(2025, 8, 17))

In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-08-18
yesterday: 2025-08-15


In [7]:
format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

### New dividend stock

In [4]:
name = 'LHK'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [5]:
sqlIns = """
INSERT INTO dividend
VALUES('LHK',0,0.20,0,0.45,0.65,0,0,00,'2022-11-25','2022-12-09','',1)
"""
rp = const.execute(sqlIns)
rp.rowcount

1

### Start of Update dividend

In [9]:
name = 'TFFIF'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TFFIF,0.1039,0.1192,0.1176,0.1189,0.4596,"20,000",2025-05-28,2025-06-16,,0


In [11]:
from sqlalchemy import text

def update_dividend1(q3, XDATE, PAIDDATE, actual, name):
    sql = text("UPDATE dividend SET q3 = :q3, dividend = q1+q2+q3+q4, XDATE = :xdate, PAIDDATE = :pdate, actual = :actual WHERE name = :name")
    rp = const.execute(sql, {"q3": q3, "xdate": XDATE, "pdate": PAIDDATE, "actual": actual, "name": name})
    return "Records updated = " + str(rp.rowcount)

In [13]:
q3 = 0.1128
XDATE = '2025-08-28'
PAIDDATE = '2025-09-15'
actual = 1

update_dividend1(q3,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [15]:
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TFFIF,0.1039,0.1128,0.1176,0.1189,0.4532,"20,000",2025-08-28,2025-09-15,,1


In [26]:
def update_dividend3(shares,q3,actual,name):
    sql = "UPDATE dividend SET shares = %s, q3 = %s, dividend=q1+q2+q3+q4, actual = %s WHERE name = '%s'"
    sql = sql % (shares, q3, actual, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [27]:
name = '3BBIF'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,3BBIF,0.1600,0.0000,0.0000,0.0000,0.1600,"120,000",2024-08-16,2024-09-23,,1


In [12]:
shares = 120_000
q3 = 0
actual = 1

In [13]:
update_dividend3(shares,q3,actual,name)

'Records updated = 1'

### Toggle actual status

In [14]:
name = 'TFFIF'
actual = 0
sqlUpd = "UPDATE dividend SET actual = %s WHERE name = '%s'"
sqlUpd = sqlUpd % (actual, name)
rp = const.execute(sqlUpd)
rp.rowcount

1

### Delete dividend record

In [15]:
sqlDel = '''
DELETE FROM dividend
WHERE name IN ("DOHOME")
'''
rp = const.execute(sqlDel)
rp.rowcount

0

### Start of output to cloud

In [12]:
sql = """
SELECT Y.NAME AS name,  Q3 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND P.date = '%s'
AND (Q3 > 0 OR Y.name IN ('GVREIT','TFFIF')) 
ORDER BY name
"""
sql = sql % yesterday 
print(sql)


SELECT Y.NAME AS name,  Q3 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND P.date = '2024-11-14'
AND (Q3 > 0 OR Y.name IN ('GVREIT','TFFIF')) 
ORDER BY name



In [13]:
cols = 'name qtrly shares amount net xdate paiddate cost_amt pct actual'.split()

In [14]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend[cols].style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,actual
0,AIMIRT,0.2150,"12,500","2,687.50","2,418.75",2024-11-26,2024-12-13,"136,250.00",1.78%,1
1,CPNREIT,0.2213,"55,000","12,171.50","10,954.35",2024-11-26,2024-12-12,"990,000.00",1.11%,1
2,DIF,0.2222,"35,000","7,777.00","6,999.30",2024-11-14,2024-12-06,"486,500.00",1.44%,1
3,GVREIT,0.1911,"60,000","11,466.00","10,319.40",2024-08-28,2024-09-12,"474,000.00",2.18%,0
4,IVL,0.1750,"7,200","1,260.00","1,134.00",2024-11-25,2024-12-11,"288,000.00",0.39%,1
5,ORI,0.0210,"50,000","1,050.00",945.00,2024-11-28,2024-12-13,"520,000.00",0.18%,1
6,PTG,0.1000,"3,600",360.00,324.00,2024-11-26,2024-12-12,"41,040.00",0.79%,1
7,RCL,0.5000,"27,000","13,500.00","12,150.00",2024-11-21,2024-12-06,"1,046,250.00",1.16%,1
8,TFFIF,0.1057,"20,000","2,114.00","1,902.60",2023-12-12,2023-12-28,"150,000.00",1.27%,0
9,WHAIR,0.2229,"50,000","11,145.00","10,030.50",2024-11-21,2024-12-17,"435,000.00",2.31%,1


In [15]:
dividend.net.sum()

60651.9

In [16]:
file_name = '18-div-24q3.xlsx'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name
xsl_file = xsl_path + file_name

dividend[cols].sort_values(['name'],ascending=[True]).to_excel(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(data_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(box_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(one_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(xsl_file, index=False)

### End of output to cloud

In [21]:
sql = 'SELECT name, q_eps, aq_eps, publish_date FROM epss WHERE year = %s-1 AND quarter = %s ORDER BY name'
sql = sql % (year, quarter)
print(sql)
eps = pd.read_sql(sql, conlt)
eps.dtypes

SELECT name, q_eps, aq_eps, publish_date FROM epss WHERE year = 2023-1 AND quarter = 3 ORDER BY name


name             object
q_eps           float64
aq_eps          float64
publish_date     object
dtype: object

In [22]:
df_merge = pd.merge(dividend, eps, on='name', how='inner')
df_merge.sort_values('name',ascending=True)

,name,qtrly,shares,xdate,paiddate,price,dividend,actual,cost_amt,amount,net,pct,q_eps,aq_eps,publish_date
0,AIMIRT,0.2230,12500,2024-08-26,2024-09-12,10.80,0.8680,1,136250.0,2787.5,2508.75,1.84,0.4277,0.9044,2022-11-14
1,CPNREIT,0.2570,55000,2024-08-26,2024-09-11,12.10,0.8758,1,990000.0,14135.0,12721.50,1.28,0.2618,0.7765,2022-11-07
2,DIF,0.2264,30000,2024-08-13,2024-09-05,9.35,0.8972,1,441000.0,6792.0,6112.80,1.39,0.2694,0.8123,2022-11-11
3,GVREIT,0.1911,60000,2024-08-28,2024-09-12,6.75,0.7833,1,474000.0,11466.0,10319.40,2.18,0.2416,0.7164,2022-08-15
4,IVL,0.2500,7200,2024-08-27,2024-09-12,26.00,0.7750,1,288000.0,1800.0,1620.00,0.56,1.4200,7.4700,2022-11-10
5,TFFIF,0.1057,20000,2023-12-12,2023-12-28,6.45,0.4297,0,150000.0,2114.0,1902.60,1.27,0.0922,0.2671,2022-08-11
6,WHAIR,0.1369,50000,2024-08-23,2024-09-24,6.65,0.5482,1,435000.0,6845.0,6160.50,1.42,0.1537,0.4642,2022-11-08
7,WHART,0.2590,20000,2024-08-20,2024-09-06,10.30,0.7735,1,246000.0,5180.0,4662.00,1.90,0.2053,0.4441,2022-11-14


In [23]:
colt = 'name qtrly shares amount net publish_date xdate paiddate q_eps aq_eps'.split()

In [24]:
df_merge[colt].sort_values('publish_date',ascending=True)

,name,qtrly,shares,amount,net,publish_date,xdate,paiddate,q_eps,aq_eps
5,TFFIF,0.1057,20000,2114.0,1902.60,2022-08-11,2023-12-12,2023-12-28,0.0922,0.2671
3,GVREIT,0.1911,60000,11466.0,10319.40,2022-08-15,2024-08-28,2024-09-12,0.2416,0.7164
1,CPNREIT,0.2570,55000,14135.0,12721.50,2022-11-07,2024-08-26,2024-09-11,0.2618,0.7765
6,WHAIR,0.1369,50000,6845.0,6160.50,2022-11-08,2024-08-23,2024-09-24,0.1537,0.4642
4,IVL,0.2500,7200,1800.0,1620.00,2022-11-10,2024-08-27,2024-09-12,1.4200,7.4700
2,DIF,0.2264,30000,6792.0,6112.80,2022-11-11,2024-08-13,2024-09-05,0.2694,0.8123
0,AIMIRT,0.2230,12500,2787.5,2508.75,2022-11-14,2024-08-26,2024-09-12,0.4277,0.9044
7,WHART,0.2590,20000,5180.0,4662.00,2022-11-14,2024-08-20,2024-09-06,0.2053,0.4441


In [25]:
file_name = '15-div-23q3.xlsx'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name
xsl_file = xsl_path + file_name

df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(output_file, index=False)
df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(data_file, index=False)
df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(box_file, index=False)
df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(one_file, index=False)
df_merge[colt].sort_values(['name'],ascending=[True]).to_excel(xsl_file, index=False)

In [ ]:
const.commit()
const.close()